In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
tf.__version__

In [ ]:
#definning the basemodel
base_model = tf.keras.applications.InceptionV3(include_top=False, weights = 'imagenet')
base_model.summary()

In [ ]:
#big ass cnn
#selecting the layers for output extraction
layer_names = ['mixed3', 'mixed5', 'mixed7', 'mixed9'] # Increased layers for more patterns
layer = [base_model.get_layer(name).output for name in layer_names]

deep_dream_model = tf.keras.Model(inputs = base_model.input, outputs = layer)

In [ ]:
#loading the image
image_url = 'https://i.pinimg.com/736x/d1/f0/55/d1f055ce5a33f65258a3e664f4583dd6.jpg'
image_path = tf.keras.utils.get_file(
    origin=image_url,
    fname='deep_dream_image.jpg')

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(225, 375))

plt.imshow(image)
image = tf.keras.preprocessing.image.img_to_array(image)

#applying neccesary transformation fr model
image = tf.keras.applications.inception_v3.preprocess_input(image)





In [ ]:
#getting activation functions working

image_batch = tf.expand_dims(image, axis=0)
activations = deep_dream_model.predict(image_batch)
activations[0].shape, activations[1].shape

In [ ]:
#calculating the loss
def calc_loss_func(image, network):
  image_batch = tf.expand_dims(image, axis=0)
  activations = network(image_batch) # Changed .predict to direct call

  losses = []
  for actvn in activations:
    loss = tf.math.reduce_mean(actvn)
    losses.append(loss)
    # Removed print statements that cause AttributeError inside tf.function
    # print(losses)
    # print(np.shape(losses))

  return tf.reduce_sum(losses)

In [ ]:
#defining the deep dreaming algo
@tf.function
def deep_dream(network, image, learning_rate):
  with tf.GradientTape() as tape:
    tape.watch(image)
    loss = calc_loss_func(image, network)

  gradient = tape.gradient(loss, image)
  gradient /= tf.math.reduce_std(gradient)
  image = image + gradient * learning_rate

  return loss, tf.clip_by_value(image, -1, 1)

In [ ]:
def inverse_transform(image):
  image = 255 * (image + 1.0) / 2.0
  return tf.cast(image, tf.uint8)


def run_deep_dream(image, network, steps, learning_rate):
  for epoch in range(steps):
    loss, image = deep_dream(network, image, learning_rate)

    if epoch % 100 == 0:
      plt.figure(figsize=(12,12))
      display_image = inverse_transform(image) # Create a displayable image
      plt.imshow(display_image) # Use the displayable image
      plt.show()
      print(f"Epoch: {epoch}, Loss: {loss}")
  return image

In [ ]:
#deep dreaming an image
image = run_deep_dream(image, deep_dream_model, 8000, 0.0001)

In [ ]:
#loading the image
image_url = 'https://cdn.britannica.com/17/83817-050-67C814CD/Mount-Everest.jpg'
image_path = tf.keras.utils.get_file(
    origin=image_url,
    fname='deep_dream_new.jpg')

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(225, 375))

plt.imshow(image)
image = tf.keras.preprocessing.image.img_to_array(image)

#applying neccesary transformation fr model
image = tf.keras.applications.inception_v3.preprocess_input(image)

image = run_deep_dream(image, deep_dream_model, 8000, 0.01)





In [ ]:
#loading the image
image_url = 'https://images.theconversation.com/files/623503/original/file-20241004-16-zlyl06.jpg?ixlib=rb-4.1.0&q=45&auto=format&w=754&fit=clip'
image_path = tf.keras.utils.get_file(
    origin=image_url,
    fname='deep_dream_new1.jpg')

image = tf.keras.preprocessing.image.load_img(image_path, target_size=(225, 375))

plt.imshow(image)
image = tf.keras.preprocessing.image.img_to_array(image)

#applying neccesary transformation fr model
image = tf.keras.applications.inception_v3.preprocess_input(image)

image = run_deep_dream(image, deep_dream_model, 100, 0.001)



